In [1]:
# Eric's modification

import os
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from torch.utils.data import TensorDataset, DataLoader
from os import path

from src.models import *
from src.methods import *
from src.utils import *

data_path = os.path.join(os.getcwd(), 'project_data\\')


USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [2]:
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")
person_train_valid = np.load(data_path+"person_train_valid.npy")
X_train_valid = np.load(data_path+"X_train_valid.npy")
y_train_valid = np.load(data_path+"y_train_valid.npy")
person_test = np.load(data_path+"person_test.npy")
#:\Users\pluto\Desktop\2024_winter\ECE_147\final_project\ECE247-Group-Project\src\project_data\y_test.npy

In [3]:
y_train_valid-=769
y_test-=769

In [4]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [5]:
batch_size = 64

In [6]:
X_train_valid = torch.tensor(X_train_valid, dtype=torch.float)
y_train_valid = torch.tensor(y_train_valid, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)

num_train = int(0.8 * len(train_valid_dataset))
num_valid = len(train_valid_dataset) - num_train
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [num_train, num_valid])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
writer = SummaryWriter('runs/ConvNet')
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

train(model, train_loader, valid_loader, criterion, optimizer, device=device, writer=writer, epochs=50)

test_accuracy = evaluate(model, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')
writer.add_scalar('test_accuracy',test_accuracy)
writer.close()

  0%|          | 0/50 [00:00<?, ?it/s]

 20%|██        | 10/50 [00:13<00:51,  1.28s/it]

Epoch [10/50], Loss: 3.7005, Validation Accuracy: 30.26%


 40%|████      | 20/50 [00:26<00:38,  1.28s/it]

Epoch [20/50], Loss: 2.2396, Validation Accuracy: 36.17%


...

Epoch [30/50], Loss: 1.6393, Validation Accuracy: 37.59%


 80%|████████  | 40/50 [00:51<00:12,  1.30s/it]

Epoch [40/50], Loss: 1.0932, Validation Accuracy: 45.15%


 96%|█████████▌| 48/50 [01:03<00:02,  1.32s/it]

Early stopping!
Test Accuracy: 46.95%


In [8]:
lstm = LSTMModel(input_size=1000)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

lstm = lstm.to(device)
train(lstm, train_loader, valid_loader, criterion, optimizer, device=device)

test_accuracy = evaluate(lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 50%|█████     | 10/20 [00:04<00:03,  2.75it/s]

Epoch [10/20], Loss: 0.1269, Validation Accuracy: 29.55%


100%|██████████| 20/20 [00:07<00:00,  2.61it/s]

Epoch [20/20], Loss: 0.0884, Validation Accuracy: 31.44%
Test Accuracy: 27.77%


In [17]:
cnn_lstm = CNN_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

cnn_lstm = cnn_lstm.to(device)
train(cnn_lstm, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(cnn_lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 10%|█         | 10/100 [00:12<01:51,  1.24s/it]

Epoch [10/100], Loss: 1.1203, Validation Accuracy: 41.84%


 20%|██        | 20/100 [00:24<01:39,  1.24s/it]

Epoch [20/100], Loss: 0.5498, Validation Accuracy: 45.39%


 28%|██▊       | 28/100 [00:35<01:31,  1.26s/it]

Early stopping!
Test Accuracy: 51.24%


In [6]:
   
rnn = RNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

rnn = rnn.to(device)
train(rnn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(rnn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 10%|█         | 10/100 [00:03<00:34,  2.61it/s]

Epoch [10/100], Loss: 1.2654, Validation Accuracy: 29.79%


 20%|██        | 20/100 [00:07<00:32,  2.44it/s]

Epoch [20/100], Loss: 0.9364, Validation Accuracy: 28.61%


 20%|██        | 20/100 [00:08<00:33,  2.39it/s]

Early stopping!
Test Accuracy: 20.99%


In [12]:
eegn = EEGNet_Modified()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(eegn.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

eegn = eegn.to(device)
train(eegn, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(eegn, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\pluto\anaconda3\envs\kaggle\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ..\aten\src\ATen\native\Convolution.cpp:1041.)
  return F.conv2d(input, weight, bias, self.stride,
 10%|█         | 10/100 [00:10<01:04,  1.39it/s]

Epoch [10/100], Loss: 0.8731, Validation Accuracy: 61.23%


 20%|██        | 20/100 [00:17<00:54,  1.47it/s]

Epoch [20/100], Loss: 0.7346, Validation Accuracy: 67.38%


...

Epoch [80/100], Loss: 0.4772, Validation Accuracy: 74.23%


 90%|█████████ | 90/100 [01:05<00:06,  1.45it/s]

Epoch [90/100], Loss: 0.4289, Validation Accuracy: 74.94%


100%|██████████| 100/100 [01:12<00:00,  1.38it/s]

Epoch [100/100], Loss: 0.4515, Validation Accuracy: 75.41%
Test Accuracy: 71.33%
